<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/CheckVital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Hemodialysis.csv', encoding='latin1')

In [3]:
import pandas as pd
vital = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/vitalsign.csv')
vital.shape

(9213497, 7)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

# Remove patient UIDs from visit['PatientUID'] that are not in the list
vital = vital[~vital['PatientUID'].isin(patient_uid_list)]
vital.shape

(8099274, 7)

In [5]:
# Specify the columns for unique value counting
columns_to_count = ['OrderItemName', 'ResultItemName']

# Count unique values in the specified columns
unique_counts = vital[columns_to_count].value_counts().reset_index()

In [6]:
unique_counts.columns = ['OrderItemName','ResultItemName', 'Count']

In [7]:
selected_items = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA']
filtered_items = unique_counts[unique_counts['ResultItemName'].isin(selected_items)]
filtered_items

,OrderItemName,ResultItemName,Count
0,Vital Signs Record,Systolic BP,1221173
1,Vital Signs Record,Diastolic BP,1220095
7,Vital Signs Record,BMI,558791
8,Vital Signs Record,BSA,558502


In [8]:
#filtered_items.to_excel('VitalFile.xlsx')

## **Slope**

> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**




In [9]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/Slope.xlsx')

In [10]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]

df = pd.DataFrame(data)

# Sorting the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(['PatientUID', 'PatientVisitUID'])

# Adding a NextSlope column
df['Slopes'] = df.groupby('PatientUID')['Slopes'].shift(-1)

data = df.dropna()

In [11]:
data

,PatientUID,PatientVisitUID,Slopes
0,2299,3386255,-7.158596
1,2299,4546471,5.550691
2,2299,5812902,-6.063710
3,2299,6947220,-12.812245
4,2299,8186767,28.287500
...,...,...,...
73378,3319961,9270330,-4.256753
73380,3319971,9270331,-0.692078
73382,3320048,9242721,7.217875
73384,3320050,9257281,-32.794557


In [12]:
vital.shape

(8099274, 7)

**เก็บแค่ visit ที่อยู่ในสโลป**

In [15]:
visitid = data['PatientVisitUID'].unique().tolist()
check = vital[vital['PatientVisitUID'].isin(visitid)]
check

,LabNo,PatientUID,PatientVisitUID,OrderItemName,ResultItemName,ResultValue,CWhen
158,1,24513,4488192,Vital Signs Record,Diastolic BP,86,2560-09-13 09:32:13.743
159,2,24513,4488192,Vital Signs Record,Diastolic BP,76,2560-09-13 10:41:42.397
160,1,24513,4488192,Vital Signs Record,Systolic BP,154,2560-09-13 09:32:13.743
161,2,24513,4488192,Vital Signs Record,Systolic BP,159,2560-09-13 10:41:42.397
180,1,24513,5791363,Vital Signs Record,Diastolic BP,77,2561-12-03 09:02:55.493
...,...,...,...,...,...,...,...
9183476,7,3321289,9274735,Vital Signs Record,Pulse Rate,116,2565-03-18 01:10:55.633
9183477,8,3321289,9274735,Vital Signs Record,Pulse Rate,124,2565-03-18 13:57:16.750
9183478,9,3321289,9274735,Vital Signs Record,Pulse Rate,124,2565-03-19 00:00:44.540
9183479,1,3321289,9274735,Vital Signs Record,Temperature,36,2565-03-15 16:58:59.983


In [17]:
len(visitid)

52620

In [16]:
check['PatientVisitUID'].nunique()

45883

# **WideTable**


    this is visitID that had dup value
    but we select only latest value

In [18]:
value = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA']
filtered = vital[vital['ResultItemName'].isin(value)]

grouped_check = filtered.groupby(['PatientUID', 'PatientVisitUID', 'ResultItemName']).agg({'ResultValue': 'last'}).reset_index()
grouped_check

,PatientUID,PatientVisitUID,ResultItemName,ResultValue
0,2299,0,BMI,21.04
1,2299,0,BSA,1.38
2,2299,0,Diastolic BP,76
3,2299,0,Systolic BP,130
4,2299,2024049,BMI,20.17
...,...,...,...,...
2570704,3437526,10191122,Systolic BP,155
2570705,3437526,10261871,BMI,23.88
2570706,3437526,10261871,BSA,1.72
2570707,3437526,10261871,Diastolic BP,65


In [19]:
pivoted_table = grouped_check.pivot_table(index=['PatientUID', 'PatientVisitUID'], columns='ResultItemName', values='ResultValue', aggfunc='last')
pivoted_table.reset_index(inplace=True)

In [31]:
grouped_check[grouped_check['PatientVisitUID']==4916314]

,PatientUID,PatientVisitUID,ResultItemName,ResultValue
76,2299,4546471,BMI,20.27
77,2299,4546471,BSA,1.38
78,2299,4546471,Diastolic BP,64
79,2299,4546471,Systolic BP,105


In [30]:
vital[vital['PatientVisitUID']==4916314]

,LabNo,PatientUID,PatientVisitUID,OrderItemName,ResultItemName,ResultValue,CWhen


In [22]:
result_table = pd.merge(data, pivoted_table, on=['PatientUID', 'PatientVisitUID'], how='left')
result_table

,PatientUID,PatientVisitUID,Slopes,BMI,BSA,Diastolic BP,Systolic BP
0,2299,3386255,-7.158596,20.8,1.39,75,127
1,2299,4546471,5.550691,20.27,1.38,64,105
2,2299,5812902,-6.063710,20.72,1.38,76,138
3,2299,6947220,-12.812245,20.27,1.36,66,134
4,2299,8186767,28.287500,19.05,1.33,57,103
...,...,...,...,...,...,...,...
52615,3319961,9270330,-4.256753,29.75,1.88,78,134
52616,3319971,9270331,-0.692078,25.97,1.63,57,127
52617,3320048,9242721,7.217875,NaN,NaN,87,130
52618,3320050,9257281,-32.794557,NaN,NaN,83,137


In [23]:
result_table.isnull().sum()

PatientUID             0
PatientVisitUID        0
Slopes                 0
BMI                15327
BSA                15336
Diastolic BP        6799
Systolic BP         6789
dtype: int64

In [24]:
result_table[result_table['PatientUID'].isnull()]

,PatientUID,PatientVisitUID,Slopes,BMI,BSA,Diastolic BP,Systolic BP


In [25]:
result_table.isnull().sum()

PatientUID             0
PatientVisitUID        0
Slopes                 0
BMI                15327
BSA                15336
Diastolic BP        6799
Systolic BP         6789
dtype: int64

In [26]:
vitalDF = result_table.copy()

In [27]:
vitalDF.to_excel('VitalDF.xlsx')